In [1]:
import numpy as np
import matplotlib.pyplot as plt
import iDEA
import ipywidgets as widgets
from IPython.display import display
import pickle
import time
from pathlib import Path
import collections_iDEA
import collections_iDEA.multiplets as mm
import os
import scipy.special as sp

# Initialize the system
points = 231
l = 10
x = np.linspace(-l, l, points)
dx = x[1] - x[0]
omega = 0.3275
v_ext = 0.5 * omega**2 * (x**2)
v_int = np.zeros([len(x), len(x)])
qho_single = iDEA.system.System(x, v_ext, v_int, electrons="u")
qho_double = iDEA.system.System(x, v_ext, v_int, electrons="ud")

states_small_int = {}
states_zero_int = {}
occupied_info = []

directory = Path("states")

for i in range(0, 1):
    file_path = directory / f"qho_small_int_state_{i}_1e-11.pkl"
    try:
        with file_path.open("rb") as file:
            states_small_int[i] = pickle.load(file)
    except FileNotFoundError:
        print(f"File state_{i}.pkl not found.")
    except Exception as e:
        print(f"An error occurred while loading state_{i}.pkl: {e}")

for i in range(0, 51):
    file_path = directory / f"qho_zero_int_state_{i}.pkl"
    try:
        with file_path.open("rb") as file:
            states_zero_int[i] = pickle.load(file)
    except FileNotFoundError:
        print(f"File state_{i}.pkl not found.")
    except Exception as e:
        print(f"An error occurred while loading state_{i}.pkl: {e}")

# file_path = "occupied_info_231.pkl"
# with open(file_path, "rb") as file:
#     occupied_info = pickle.load(file)


with open("result_200.pkl", "rb") as file:
    index_list = pickle.load(file)

print(index_list)

In [2]:
analytic_prob_densities = []

def wave_function(index, omega, x):
    exponent = -(omega * (x**2)) / (2)
    t = np.sqrt(1 / ((2**index) * sp.factorial(index))) * pow((omega/np.pi), 0.25) * np.exp(exponent) * sp.eval_hermite(index, np.sqrt(omega)*x)

    return t

for i in range(48):
    index_k = index_list.up_indices[i]
    index_j = index_list.down_indices[i]

    pd_analytic_1 = (np.abs(wave_function(index_k, omega, x)))**2 + (np.abs(wave_function(index_j, omega, x)))**2
    analytic_prob_densities.append(pd_analytic_1)

In [3]:
collections_iDEA.analysis.states_density_comparison(states_zero_int, qho_double, analytic_prob_densities)

interactive(children=(IntSlider(value=0, description='k:', max=50), Output()), _dom_classes=('widget-interact'…